<a href="https://colab.research.google.com/github/DragonFarts/Video-EZY/blob/main/AnimateDiff%2BIPadapter%2BMotionLoRA_(AP123_AnimateImage).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers accelerate transformers xformers peft gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.

In [ ]:
import gradio as gr
import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, DDIMScheduler
from diffusers.utils import export_to_gif
import random

def generate_gif(image, animation_type):
    # Load the motion adapter
    adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16)

    # Load SD 1.5 based finetuned model
    model_id = "SG161222/Realistic_Vision_V6.0_B1_noVAE"
    pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch.float16)

    # Scheduler setup
    scheduler = DDIMScheduler(
        clip_sample=False,
        beta_start=0.00085,
        beta_end=0.012,
        beta_schedule="linear",
        timestep_spacing="trailing",
        steps_offset=1
    )
    pipe.scheduler = scheduler

    # Enable memory savings
    pipe.enable_vae_slicing()
    pipe.enable_model_cpu_offload()

    # Load ip_adapter
    pipe.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")

    # Load the selected motion adapter
    pipe.load_lora_weights(f"guoyww/animatediff-motion-lora-{animation_type}", adapter_name=animation_type)

    # Generate a random seed
    seed = random.randint(0, 2**32 - 1)
    prompt = "best quality, high quality, trending on artstation"

    # Set adapter weights for the selected adapter
    adapter_weight = [0.75]

    pipe.set_adapters([animation_type], adapter_weights=adapter_weight)

    # Generate GIF
    output = pipe(
        prompt=prompt,
        num_frames=16,
        guidance_scale=7.5,
        num_inference_steps=30,
        ip_adapter_image=image,
        generator=torch.Generator("cpu").manual_seed(seed),
    )
    frames = output.frames[0]

    gif_path = "output_animation.gif"
    export_to_gif(frames, gif_path)
    return gif_path

# Gradio interface
iface = gr.Interface(
    fn=generate_gif,
    inputs=[gr.Image(type="pil"), gr.Radio(["zoom-out", "tilt-up", "pan-left"])],
    outputs=gr.Image(type="pil", label="Generated GIF"),
    title="AnimateDiff + IP Adapter Demo",
    description="Upload an image and select an motion module type to generate a GIF!"
)

iface.launch(debug=True,share=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a06122dfa5712d9969.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]